# Read me!!

Hello, welcome to my notebook!!

This notebook is inline with my previous notebook ([A way you can handle dataset, simply]).

Furthermore, you will learn about predicting through deep learning model.

We divided the dataset into dummy cluster and non-dummy cluster because of this notebook.

The key point is parallel strategy which uses parallel layers in training the dummy and non-dummy cluster, respectively.

How can we do this strategy?? Please take a look model discussion, carefully.

If you have any questions, please leave the comments.

I hope you to gain more imformation about data handling, DNN, CNN, and etc..

## **Knowledge can be improved by being shared.**

Please upvote!!


## [You can learn more skills for handling dataset or neural network.]

### [A way you can handle dataset, simply] - Tabular Playground Series - Dec 2021 (best 0.95583 score!!)
 - https://www.kaggle.com/pythonash/a-way-you-can-handle-dataset-simply#It-has-done!!

### [How to use csv and img at the same time] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/how-to-use-csv-and-img-at-the-same-time

### [Parallel combination DNN with CNN] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/parallel-dnn-and-cnn-network-for-beginners
 
### [Image data handling without memory exploded] - Pawpularity Contest
 - https://www.kaggle.com/pythonash/how-to-handle-dataset-for-beginners

### [Data handling & Deep learning] - Titanic competition (Top 7%)
 - https://www.kaggle.com/pythonash/how-to-handle-raw-dataset-and-analyze-with-dl
 
### [Deep learning model with SeLU activation function] - Titanic competition
- https://www.kaggle.com/pythonash/selu-activation-function-in-dl

### [Preparing a completed dataset with proper imputation method] - Titanic competition
 - https://www.kaggle.com/pythonash/making-completed-dataset

**Let's start!**

# Data loading and handling will be skipped.

The handling procedure is shown as in my previous notebook (see "A way you can handle dataset, simply).

Just implement the code for preparing dataset.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold as SK
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score



train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')

test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')

submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')

def reduce_mean_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] =='int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min> np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    
    if verbose:
        print('Memory usage is decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem)/ start_mem))
    
    return df


train = reduce_mean_usage(train)
test = reduce_mean_usage(test)

# Use sample method with frac =1 .
train = train.sample(frac=1).reset_index(drop=True)

train_x = train.drop(['Id','Cover_Type'],axis=1)

test_x = test.drop(['Id'],axis=1)

train_y = train['Cover_Type']

drop_5_index = train_y[train_y == 5].index[0]

train_x = train_x.drop([drop_5_index],axis=0).reset_index().drop(['index'],axis=1)
train_y = train_y.drop([drop_5_index],axis=0).reset_index().drop(['index'],axis=1)

scaler = MinMaxScaler()
train_df = scaler.fit_transform(train_x)
train_df = pd.DataFrame(train_df)
train_df.columns = train_x.columns

test_df = pd.DataFrame(scaler.transform(test_x))
test_df.columns = test_x.columns

train_non_dummy, train_dummy = train_df.columns[:10], train_df.columns[10:]

train_df_dummy = train_df[train_dummy]
train_df_non_dummy = train_df[train_non_dummy]

test_df_dummy = test_df[train_dummy]
test_df_non_dummy = test_df[train_non_dummy]

train=[]
test=[]
train_x=[]
test_x=[]
train_df=[]
test_df=[]

Encoder = LabelEncoder()

y_encoded = Encoder.fit_transform(train_y)
train_y = []



# Deep learning with parallel strategy!!

We use DNN model for predicting the result with parallel strategy.

Parallel startegy means that we will design two structures for dummy variables and non-dummy variables, respectively.

And we will concatenate the two structures with one output.

From my idea, you will get at least 95% accuracy.

*Note*:

> We will use elu as activation function and 'he_normal' as kerner initializer to this model.

> It will be helpful for rapid convergence and commonly used as a useful way for training.

In [ ]:
# Set input two input layers for dummy and non-dummy.
INPUT = tf.keras.layers.Input(shape = train_df_non_dummy.shape[1:], name = 'Input')
INPUT_dummy = tf.keras.layers.Input(shape = train_df_dummy.shape[1:], name = 'Input_Dummy')

# Set dense layer for non-dummy variable.
dense1 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal', name = 'Dense1')(INPUT)
dense2 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense2')(dense1)
dense3 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense3')(dense2)
dense4 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense4')(dense3)
dense5 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense5')(dense4)
dense6 = tf.keras.layers.Dense(300, activation='elu',  kernel_initializer = 'he_normal',name = 'Dense6')(dense5)
dense_dropout = tf.keras.layers.Dropout(0.5, name = 'Dropout')(dense6)

# Set dense layer for dummy variable.
dummy_dense1 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense1')(INPUT_dummy)
dummy_dense2 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense2')(dummy_dense1)
dummy_dense3 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense3')(dummy_dense2)
dummy_dense4 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense4')(dummy_dense3)
dummy_dense5 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense5')(dummy_dense4)
dummy_dense6 = tf.keras.layers.Dense(200, activation = 'elu', kernel_initializer = 'he_normal',name = 'Dummy_Dense6')(dummy_dense5)
dummy_dense_dropout = tf.keras.layers.Dropout(0.5, name = 'Dummy_Dropout')(dummy_dense6)

# Concatenating the two structures.
connect = tf.keras.layers.Concatenate(axis=1, name='Connection')([dense_dropout, dummy_dense_dropout])

# Set dense layer for combined structure.
connect_dense1 = tf.keras.layers.Dense(100, activation = 'elu', kernel_initializer = 'he_normal',name = 'Connect_Dense1')(connect)
connect_dense2 = tf.keras.layers.Dense(100, activation = 'elu', kernel_initializer = 'he_normal',name = 'Connect_Dense2')(connect_dense1)
connect_dense3 = tf.keras.layers.Dense(100, activation = 'elu', kernel_initializer = 'he_normal',name = 'Connect_Dense3')(connect_dense2)
connect_dropout = tf.keras.layers.Dropout(0.5, name = 'Connect_Dropout')(connect_dense3)

# Set the output layer
OUTPUT = tf.keras.layers.Dense(6, activation = 'softmax', name = 'Output')(connect_dropout)

# Set your model
model = tf.keras.Model(inputs = [INPUT, INPUT_dummy], outputs = [OUTPUT], name = 'pythonash_model')

## Model summary

> You can identify what the structure is.

In [ ]:
model.summary()

## Plotting your model

> You can see the structure in visualiation.

> As you can see, your model will be concatenated.

In [ ]:
tf.keras.utils.plot_model(model, show_shapes = True, show_layer_names=True, rankdir='TB')

## Set hyper parameters

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.002,
    decay_steps=10000,
    decay_rate=0.98
)
opt = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
epoch_number = 20
check_pt = tf.keras.callbacks.ModelCheckpoint('pythonash_model.h5', save_best_only = True, verbose = 2)

## Compile your model and fit

In [ ]:
model.compile(loss ='sparse_categorical_crossentropy', optimizer = opt, metrics = ['sparse_categorical_accuracy'])
model.fit(
x= [train_df_non_dummy, train_df_dummy], y = y_encoded, validation_split = 0.1 , batch_size = 1000, validation_batch_size = 1000,
callbacks = [check_pt], verbose =2, workers = 3, epochs = epoch_number)

## Use your recorded model with best performance

> With using ModelCheckpoint in tf.keras.callbacks, we can load and use the best model which was the best performance.

In [ ]:
best_model = tf.keras.models.load_model('pythonash_model.h5')
result = best_model.predict([test_df_non_dummy, test_df_dummy])
result = Encoder.inverse_transform(np.argmax(result,axis=1))
final_result = pd.DataFrame(result)
final_result.columns =['Cover_Type']
final_result

## Submit your result!!

In [ ]:
submission['Cover_Type'] = final_result['Cover_Type']

submission.to_csv('./submission.csv', index=False)

submission

# It's your turn!!

You have many opportunities that you can change this model parameters and get your submission score.

I recommend that you change the hyper parameters such as learning_rate, batch_size, activation function, the number of neurons, layers, and so on...

If you get any helps from my notebook, please upvote!!

Fingers crossed!!